In [1]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document

import os

In [2]:
# Configurable parameters
data_dir = "./data"
chunk_size = 128
chunk_overlap = 32

# Ensure data directory exits
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Get list of files
files = os.listdir(data_dir)
print(f"List of files: {files}")

text_data_from_files = []
for file in files:
    try: 
        with open(os.path.join(data_dir, file), "r") as f:
            file_data = f.read()
        text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
            chunk_size = chunk_size,
            chunk_overlap = chunk_overlap
        )
        split_text = text_splitter.split_text(file_data)
        print("Split text: ", len(split_text))
        
        for index, chunked_text in enumerate(split_text):
            text_data_from_files.append(Document(
                page_content=chunked_text,
                metadata = {
                    "source": file, 
                    "page_content": chunked_text,
                    "title": file.split(".")[0],
                    "chunk_index": index
                }
            ))
    except Exception as e:
        print(f"Error processing file {file}: {e}")

print(f"Text data from files: ", [doc.page_content for doc in text_data_from_files])

List of files: ['product.md']
Split text:  7
Text data from files:  ['Synthetic Product Data for RAG App\nBelow is a collection of synthetic data representing imaginary products for use in a sample Retrieval-Augmented Generation (RAG) application. Each product includes a name, description, category, price, and unique identifier.\nProduct 1\n\nID: PRD-001\nName: Quantum Widget\nCategory: Electronics\nPrice: $149.99\nDescription: A cutting-edge quantum widget designed to optimize energy flow in smart home devices. Features a sleek design with voice-activated controls and compatibility with all major IoT platforms.\n\nProduct 2', 'Product 2\n\nID: PRD-002\nName: EcoGlow Lamp\nCategory: Home Decor\nPrice: $79.50\nDescription: An eco-friendly lamp that adjusts brightness based on ambient light. Made from recycled materials, it offers a warm, inviting glow and a battery life of up to 48 hours.\n\nProduct 3', 'Product 3\n\nID: PRD-003\nName: HyperFit Tracker\nCategory: Wearables\nPrice: $199.

In [3]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()
vector_store = FAISS.from_documents(text_data_from_files, embeddings)

/var/folders/fv/p4rk9t9j0vbfm_104cc1swq80000gn/T/ipykernel_27395/3143915071.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/var/folders/fv/p4rk9t9j0vbfm_104cc1swq80000gn/T/ipykernel_27395/3143915071.py:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/Users/aaronparandian/Desktop/codespaces/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IPro

In [5]:
query = "How much is AeroBrew Coffee Maker?"

retriever = vector_store.as_retriever(search_kwargs={"k": 1}) # default is 4
retriever.invoke(query)

[Document(metadata={'source': 'product.md', 'page_content': 'Product 4\n\nID: PRD-004\nName: AeroBrew Coffee Maker\nCategory: Kitchen Appliances\nPrice: $129.95\nDescription: A compact coffee maker that brews barista-quality coffee in under 2 minutes. Features customizable brew strength and a self-cleaning mechanism for easy maintenance.\n\nProduct 5\n\nID: PRD-005\nName: SkyVault Storage Device\nCategory: Tech Accessories\nPrice: $249.99\nDescription: A high-capacity, cloud-integrated storage device with advanced encryption for secure data management. Supports seamless file sharing across multiple devices.\n\nProduct 6', 'title': 'product', 'chunk_index': 3}, page_content='Product 4\n\nID: PRD-004\nName: AeroBrew Coffee Maker\nCategory: Kitchen Appliances\nPrice: $129.95\nDescription: A compact coffee maker that brews barista-quality coffee in under 2 minutes. Features customizable brew strength and a self-cleaning mechanism for easy maintenance.\n\nProduct 5\n\nID: PRD-005\nName: Sky